In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
IMG_SIZE=112
N_CHANNELS=1

In [ ]:
HEIGHT = 137
WIDTH = 236

In [ ]:
preds_dict = {
    'grapheme_root': [],
    'vowel_diacritic': [],
    'consonant_diacritic': []
}

In [ ]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder

In [ ]:
models_list = []
model = tf.keras.models.load_model('/kaggle/input/densenet121-fold-1/densenet121_fold_1_98906.h5', compile=False)
models_list.append(model)
del model
gc.collect()

In [ ]:
class TestDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, height, width, n_channels, batch_size=32):
        self._data = data
        self._list_idx = np.arange(len(data))
        self._batch_size = batch_size
        self.height = height
        self.width = width
        self.n_channels = n_channels
        self._indices = np.arange(len(self._list_idx))
        
    def __len__(self):
        return int(np.ceil(len(self._data) / float(self._batch_size)))
    
    def __getitem__(self, index):
        batch_idx = self._indices[index*self._batch_size:(index+1)*self._batch_size]
        Data = np.empty((len(batch_idx), self.height, self.width, self.n_channels))
        
        for i, k in enumerate(batch_idx):
            image = self._data[k].copy().reshape(self.height, self.width, self.n_channels).astype(np.float32) / 255            
            Data[i,:, :, :] =  image
        
        return Data

In [ ]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
target=[] # model predictions placeholder
row_id=[] # row_id place holder
for i in range(4):
    df_test_img = pd.read_parquet('/kaggle/input/bengaliai-cv19/test_image_data_{}.parquet'.format(i))
    df_test_img.set_index('image_id', inplace=True)
    
    '''X_test = np.zeros((len(df_test_img), HEIGHT, WIDTH, N_CHANNELS))
    for j in range(len(df_test_img)):
        img = df_test_img.iloc[j].values.astype(np.float32).reshape(HEIGHT, WIDTH, N_CHANNELS) / 255
        X_test[j] = img'''
    
    #X_test = df_test_img.values.reshape(-1, HEIGHT, WIDTH, N_CHANNELS).astype(np.float32) / 255
    
    test_datagen = TestDataGenerator(df_test_img.values, height=HEIGHT, width=WIDTH, n_channels=N_CHANNELS)
    
    preds_0_list = []
    preds_1_list = []
    preds_2_list = []
    for model in models_list:
        preds = model.predict_generator(test_datagen)
        preds_0_list.append(preds[0])
        preds_1_list.append(preds[1])
        preds_2_list.append(preds[2])
    mean_preds_0 = np.mean(preds_0_list, axis=0)
    mean_preds_1 = np.mean(preds_1_list, axis=0)
    mean_preds_2 = np.mean(preds_2_list, axis=0)
    preds = [mean_preds_0, mean_preds_1, mean_preds_2]

    for z, p in enumerate(preds_dict):
        preds_dict[p] = np.argmax(preds[z], axis=1)

    for k,_id in enumerate(df_test_img.index.values):  
        for q,comp in enumerate(components):
            id_sample=_id+'_'+comp
            row_id.append(id_sample)
            target.append(preds_dict[comp][k])
    del df_test_img, test_datagen, preds_0_list, preds_1_list, preds_2_list, mean_preds_0, mean_preds_1, mean_preds_2, preds 
    gc.collect()

del preds_dict
gc.collect()
    
df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)
del row_id, target
gc.collect()
df_sample.to_csv('submission.csv',index=False)
df_sample.head()